In [ ]:
# Setting options for the plots
%matplotlib inline
%config InlineBackend.figure_formats={'retina', 'svg'}
%config InlineBackend.rc={'savefig.dpi': 150}

In [ ]:
## filter out warnings from third-party libraries to prevent them
# from showing up in the notebooks in multiple places
import warnings

# warning from shap about tqdm 
from tqdm import TqdmExperimentalWarning
warnings.filterwarnings("ignore", 
                        category=TqdmExperimentalWarning, 
                        message="Using `tqdm.autonotebook.tqdm` .*", 
                        module="shap.explainers._linear")

# warning from matplotlib -> seaborn when figure layouts are changed to "tight"
warnings.filterwarnings("ignore", 
                        category=UserWarning, 
                        message=".*figure layout.*", 
                        module="seaborn.axisgrid")# warning from seaborn about a deprecated method in pandas
warnings.filterwarnings("ignore", 
                        category=FutureWarning, 
                        message=".*is_categorical_dtype is deprecated.*", 
                        module="seaborn")



In [ ]:
import itertools
import json
import os
import re
import pickle
import platform
import time

from collections import defaultdict as dd
from functools import partial
from os.path import abspath, dirname, exists, join
from string import Template

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from matplotlib import pyplot as plt

from IPython import sys_info
from IPython.display import display, HTML, Image, Javascript, Markdown, SVG

from rsmtool.utils.files import (get_output_directory_extension,
                                 parse_json_with_comments)
from rsmtool.utils.notebook import (float_format_func,
                                    int_or_float_format_func,
                                    bold_highlighter,
                                    color_highlighter,
                                    show_thumbnail)

from rsmtool.reader import DataReader
from rsmtool.writer import DataWriter
from rsmtool.version import VERSION as rsmtool_version

# turn off interactive plotting
plt.ioff()

In [ ]:
rsm_report_dir = os.environ.get('RSM_REPORT_DIR', None)
if rsm_report_dir is None:
    rsm_report_dir = os.getcwd()

rsm_environ_config = join(rsm_report_dir, '.environ.json')
if not exists(rsm_environ_config):
    raise FileNotFoundError('The file {} cannot be located. '
                            'Please make sure that either (1) '
                            'you have set the correct directory with the `RSM_REPORT_DIR` '
                            'environment variable, or (2) that your `.environ.json` '
                            'file is in the same directory as your notebook.'.format(rsm_environ_config))
    
environ_config = parse_json_with_comments(rsm_environ_config)

<style type="text/css">
  div.prompt.output_prompt { 
    color: white; 
  }
  
  span.highlight_color {
    color: red;
  }
  
  span.highlight_bold {
    font-weight: bold;  
  }
    
  @media print {
    @page {
      size: landscape;
      margin: 0cm 0cm 0cm 0cm;
    }

    * {
      margin: 0px;
      padding: 0px;
    }

    #toc {
      display: none;
    }

    span.highlight_color, span.highlight_bold {
        font-weight: bolder;
        text-decoration: underline;
    }

    div.prompt.output_prompt {
      display: none;
    }
    
    h3#Python-packages, div#packages {
      display: none;
  }
</style>

In [ ]:
# NOTE: you will need to set the following manually
# if you are using this notebook interactively.
summary_id = environ_config.get('SUMMARY_ID')
description = environ_config.get('DESCRIPTION')
jsons = environ_config.get('JSONS')
output_dir = environ_config.get('OUTPUT_DIR')
use_thumbnails = environ_config.get('USE_THUMBNAILS')
file_format_summarize = environ_config.get('FILE_FORMAT')

# groups for subgroup analysis.
groups_desc = environ_config.get('GROUPS_FOR_DESCRIPTIVES') 
groups_eval = environ_config.get('GROUPS_FOR_EVALUATIONS') 

# javascript path
javascript_path = environ_config.get("JAVASCRIPT_PATH")

# Summary Report 

In [ ]:
# initialize id generator for thumbnails
id_generator = itertools.count(1)

In [ ]:
with open(join(javascript_path, "sort.js"), "r", encoding="utf-8") as sortf:
    display(Javascript(data=sortf.read()))

In [ ]:
# load the information about all models
model_list = []
for (json_file, experiment_name) in jsons:
    model_config = json.load(open(json_file))
    model_id = model_config['experiment_id']
    model_name = experiment_name if experiment_name else model_id
    model_csvdir = dirname(json_file)
    model_file_format = get_output_directory_extension(model_csvdir, model_id)
    model_list.append((model_id, model_name, model_config, model_csvdir, model_file_format))


In [ ]:
Markdown("This report presents the analysis for **{}**: {} \n ".format(summary_id, description))


In [ ]:
HTML(time.strftime('%c'))

In [ ]:
# get a matched list of model ids and descriptions
models_and_desc = zip([model_name for (model_id, model_name, config, csvdir, model_file_format) in model_list],
                      [config['description'] for (model_id, model_name, config, csvdir, file_format) in model_list])
model_desc_list = '\n\n'.join(['**{}**: {}'.format(m, d) for (m, d) in models_and_desc])

Markdown("The report compares the following models: \n\n {}".format(model_desc_list))

In [ ]:
if use_thumbnails:
    display(Markdown("""***Note: Images in this report have been converted to """
                     """clickable thumbnails***"""))

In [ ]:
%%html
<div id="toc"></div>